<a href="https://colab.research.google.com/github/Tirta2241720045/PCVK_Ganjil_2024/blob/main/Week12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nama: Tirta Nurrochman Bintang Prawira
# NIM: 2241720045
# Kelas: TI-3A
# No Absen: 27

## Langkah 1 - Import Library

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

## Langkah 2 - Pra Pengolahan Data

In [5]:
# Training data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],  # Range diperlebar
    channel_shift_range=20,    # Ditingkatkan dari 15 ke 20
    shear_range=0.2,          # Ditambahkan shear transformation
    fill_mode='nearest'       # Menentukan strategi mengisi pixels baru
)

# Training data generator
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/Minggu12/Dataset_Daging_Hewan_Ternak/Training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Validation/Testing data generator
val_datagen = ImageDataGenerator(
    rescale=1./255,
    # Tidak perlu augmentasi untuk validation set
)

validation_set = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/Minggu12/Dataset_Daging_Hewan_Ternak/Testing',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Tetap False untuk konsistensi evaluasi
)

Found 240 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


## Langkah 3 - Pembuatan Model CNN

In [6]:
import tensorflow as tf
from tensorflow.keras.regularizers import l2

# Jumlah kelas
num_classes = 3

# Inisialisasi model Sequential
cnn = tf.keras.models.Sequential()

# Block 1: Input Layer + First Convolution Block
cnn.add(tf.keras.layers.Conv2D(64, kernel_size=3, padding='same', activation='relu',
                              kernel_regularizer=l2(0.01),
                              input_shape=[224, 224, 3]))
cnn.add(tf.keras.layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

# Block 2: Second Convolution Block
cnn.add(tf.keras.layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

# Block 3: Third Convolution Block
cnn.add(tf.keras.layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

# Block 4: Fourth Convolution Block
cnn.add(tf.keras.layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.Conv2D(512, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

# Flatten layer
cnn.add(tf.keras.layers.Flatten())

# Fully Connected Layers
cnn.add(tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=l2(0.01)))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.5))

# Output Layer
cnn.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Model Compilation
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0001,  # Lower learning rate for better convergence
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08
)

# Compile model with additional metrics
cnn.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Callback untuk Learning Rate Scheduling
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# Learning Rate Scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Model checkpoint dengan format .keras yang benar
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.keras',  # Mengubah ekstensi dari .h5 ke .keras
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# List callbacks
callbacks = [lr_scheduler, early_stopping, checkpoint]

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_train_data'

## Langkah 4 - Fit CNN

In [ ]:
# Train the model
history = cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=25,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


# Langkah 5 Monitoring Hasil

In [ ]:
# Visualisasi hasil training
import matplotlib.pyplot as plt

def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Plot hasil training
plot_training_history(history)

2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.3806 - loss: -18.9890

Test Accuracy: 38.33%


NameError: name 'plt' is not defined

## Langkah 5 - Prediksi dengan 1 Citra

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

img_1 = '/content/drive/MyDrive/dataset/Minggu12/Dataset_Daging_Hewan_Ternak/single_prediction/object1.jpg'
img1 = cv.imread(img_1)
prediction = image_predict(img_1)
plt.imshow(cv.cvtColor(img1, cv.COLOR_BGR2RGB))
plt.title(prediction)
plt.axis('off')
plt.show()

NameError: name 'image_predict' is not defined